In [9]:
import os
import pandas as pd
import numpy as np
import nibabel as nib

In [31]:
hgg_img_data_train = []
counter = 0

for file in os.listdir('./data/bet_processed/'):
    if file.endswith('.nii.gz'):
        file_path = './data/bet_processed/' + file
        img = nib.load(file_path)
        array_flat = img.get_data().ravel()
        hgg_img_data_train.append(array_flat)
        
        counter += 1
        if counter%100 == 0:
            print(f"{counter} out of {len(os.listdir('./data/bet_processed/'))-1}")

100 out of 1032
200 out of 1032
300 out of 1032
400 out of 1032
500 out of 1032
600 out of 1032
700 out of 1032
800 out of 1032
900 out of 1032
1000 out of 1032


In [32]:
hgg_y = np.ones(shape=len(hgg_img_data_train), dtype=int)

In [33]:
hgg_y

array([1, 1, 1, ..., 1, 1, 1])

In [28]:
lgg_img_data_train = []
counter = 0

for file in os.listdir('./data/lgg_bet_processed/'):
    if file.endswith('.nii.gz'):
        file_path = './data/lgg_bet_processed/' + file
        img = nib.load(file_path)
        array_flat = img.get_data().ravel()
        lgg_img_data_train.append(array_flat)
        
        counter += 1
        if counter%100 == 0:
            print(f"{counter} out of {len(os.listdir('./data/lgg_bet_processed/'))-1}")

100 out of 303
200 out of 303
300 out of 303


In [29]:
lgg_y = np.zeros(shape=len(lgg_img_data_train), dtype=int)

In [30]:
lgg_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [35]:
X = hgg_img_data_train + lgg_img_data_train

In [50]:
y = np.array(list(hgg_y) + list(lgg_y))